### EXPLORACIÓN


In [57]:
import pandas as pd
import numpy as np

df_agresividad = pd.read_csv('../datasets/train_agresividad.tsv', sep='\t')
print(f"Dataset cargado: {df_agresividad.shape}")

Dataset cargado: (4950, 5)


In [58]:
print("\nPRIMERAS FILAS:")
print(df_agresividad.head())

print("\nINFORMACIÓN DE COLUMNAS:")
print(df_agresividad.columns.tolist())

print("\nVALORES ÚNICOS POR COLUMNA:")
for col in df_agresividad.columns:
    unicos = df_agresividad[col].nunique()
    print(f"{col}: {unicos} valores únicos")

    


PRIMERAS FILAS:
      id                                               text  HS  TR  AG
0  20001  Easyjet quiere duplicar el número de mujeres p...   1   0   0
1  20002  El gobierno debe crear un control estricto de ...   1   0   0
2  20003  Yo veo a mujeres destruidas por acoso laboral ...   0   0   0
3  20004  — Yo soy respetuoso con los demás, sólamente l...   0   0   0
4  20007  Antonio Caballero y como ser de mal gusto e ig...   0   0   0

INFORMACIÓN DE COLUMNAS:
['id', 'text', 'HS', 'TR', 'AG']

VALORES ÚNICOS POR COLUMNA:
id: 4950 valores únicos
text: 4950 valores únicos
HS: 2 valores únicos
TR: 2 valores únicos
AG: 2 valores únicos


In [59]:
print("\nNULOS por columna:\n", df_agresividad.isnull().sum())
print("\nDUPLICADOS en 'text':", df_agresividad.duplicated('text').sum())

print("\nDistribución AG (agresividad):")
print(df_agresividad['AG'].value_counts(), "\nPorcentaje:")
print((df_agresividad['AG'].value_counts(normalize=True) * 100).round(2))



NULOS por columna:
 id      0
text    0
HS      0
TR      0
AG      0
dtype: int64

DUPLICADOS en 'text': 0

Distribución AG (agresividad):
AG
0    3294
1    1656
Name: count, dtype: int64 
Porcentaje:
AG
0    66.55
1    33.45
Name: proportion, dtype: float64


In [60]:
print("\nDistribución HS (hate speech):")
print(df_agresividad['HS'].value_counts(), "\nPorcentaje:")
print((df_agresividad['HS'].value_counts(normalize=True) * 100).round(2))



Distribución HS (hate speech):
HS
0    2895
1    2055
Name: count, dtype: int64 
Porcentaje:
HS
0    58.48
1    41.52
Name: proportion, dtype: float64


In [61]:

print("\nCROSS TAB AG vs HS:")
print(pd.crosstab(df_agresividad['HS'], df_agresividad['AG']))


CROSS TAB AG vs HS:
AG     0     1
HS            
0   2895     0
1    399  1656


In [62]:
df_agresividad['char_len'] = df_agresividad['text'].astype(str).map(len)
df_agresividad['word_count'] = df_agresividad['text'].astype(str).map(lambda s: len(str(s).split()))
print("\nEstadísticas de longitud (chars):")
print(df_agresividad['char_len'].describe().to_string())

print("\nEstadísticas de palabra (word_count):")
print(df_agresividad['word_count'].describe().to_string())


Estadísticas de longitud (chars):
count    4950.000000
mean      129.783838
std       111.071848
min         6.000000
25%        74.000000
50%       114.000000
75%       169.000000
max      5996.000000

Estadísticas de palabra (word_count):
count    4950.000000
mean       21.376566
std        19.270690
min         1.000000
25%        12.000000
50%        19.000000
75%        28.000000
max      1057.000000


### CARGAR STOPWORDS

In [63]:
import re

with open('../datasets/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords_crudas = f.read().splitlines()

stopwords = []
for word in stopwords_crudas:
    try:
        cleaned = word.encode('latin-1').decode('utf-8', errors='ignore')
        cleaned = cleaned.strip().lower()
        if cleaned:
            stopwords.append(cleaned)
    except:
        cleaned = word.strip().lower()
        if cleaned:
            stopwords.append(cleaned)

stopwords_extra = ['http', 'https', 'www', 'com', 'twitter', 'tweet', 'rt', 'user']
stopwords.extend(stopwords_extra)

stopwords = list(set(stopwords))
print(f"Stopwords cargadas: {len(stopwords)} palabras")

Stopwords cargadas: 647 palabras


In [64]:
def preprocesar_texto(texto):
    if not isinstance(texto, str):
        return 
    
    texto = texto.lower()
    texto = re.sub(r'@\w+', '', texto)
    texto = re.sub(r'#\w+', '', texto)
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto)
    texto = re.sub(r'[^\w\sáéíóúñ]', ' ', texto)
    texto = re.sub(r'\b\d+\b', ' ', texto)
    palabras = texto.split()
    palabras_filtradas = [p for p in palabras if p not in stopwords]
    texto_limpio = ' '.join(palabras_filtradas)
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    
    return texto_limpio



In [65]:
df_agresividad['text_limpio'] = df_agresividad['text'].apply(preprocesar_texto)

print("COMPARACIÓN")
for i in range(1):
    original = df_agresividad.iloc[i]['text']
    limpio = df_agresividad.iloc[i]['text_limpio']
    print(f"  Original ({len(original)} chars): {original[:60]}...")
    print(f"  Limpio ({len(limpio)} chars): {limpio[:60]}...")

print(f"ESTADÍSTICAS DE LIMPIEZA:")
df_agresividad['longitud_limpia'] = df_agresividad['text_limpio'].apply(len)
print(f"  Longitud promedio original: {df_agresividad['char_len'].mean():.0f} chars")
print(f"  Longitud promedio limpia: {df_agresividad['longitud_limpia'].mean():.0f} chars")
print(f"  Reducción promedio: {(df_agresividad['char_len'].mean() - df_agresividad['longitud_limpia'].mean()):.0f} chars")

COMPARACIÓN
  Original (108 chars): Easyjet quiere duplicar el número de mujeres piloto' Verás t...
  Limpio (61 chars): easyjet duplicar número mujeres piloto verás tú aparcar avió...
ESTADÍSTICAS DE LIMPIEZA:
  Longitud promedio original: 130 chars
  Longitud promedio limpia: 68 chars
  Reducción promedio: 62 chars


In [66]:
from sklearn.model_selection import train_test_split

X = df_agresividad['text_limpio']  
y = df_agresividad['AG']           

print(f"X (textos limpios): {X.shape}")
print(f"y (etiquetas AG): {y.shape}")
print(f"Distribución de AG: {y.value_counts().to_dict()}")
print(f"  No agresivo (0): {y.value_counts()[0]} ({y.value_counts(normalize=True)[0]*100:.1f}%)")
print(f"  Agresivo (1): {y.value_counts()[1]} ({y.value_counts(normalize=True)[1]*100:.1f}%)")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  
)

print(f"DIVISIÓN COMPLETADA")
print(f"  Train: {X_train.shape} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Test: {X_test.shape} ({len(X_test)/len(X)*100:.1f}%)")
print(f"  Train No agresivo: {(y_train == 0).sum()} ({(y_train == 0).mean()*100:.1f}%)")
print(f"  Train Agresivo: {(y_train == 1).sum()} ({(y_train == 1).mean()*100:.1f}%)")

X (textos limpios): (4950,)
y (etiquetas AG): (4950,)
Distribución de AG: {0: 3294, 1: 1656}
  No agresivo (0): 3294 (66.5%)
  Agresivo (1): 1656 (33.5%)
DIVISIÓN COMPLETADA
  Train: (3960,) (80.0%)
  Test: (990,) (20.0%)
  Train No agresivo: 2635 (66.5%)
  Train Agresivo: 1325 (33.5%)


In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=3000,      
    min_df=3,               
    max_df=0.85,            
    stop_words=stopwords,   
    ngram_range=(1, 2),     
    analyzer='word'
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Vectorización completada")
print(f"Train TF-IDF: {X_train_tfidf.shape}")
print(f"Test TF-IDF: {X_test_tfidf.shape}")
print(f"Vocabulario: {len(vectorizer.get_feature_names_out())} palabras")

print(f"\nMUESTRA")
vocab = vectorizer.get_feature_names_out()
print(f"Primeras 20 palabras: {vocab[:20].tolist()}")

Vectorización completada
Train TF-IDF: (3960, 2754)
Test TF-IDF: (990, 2754)
Vocabulario: 2754 palabras

MUESTRA
Primeras 20 palabras: ['abajo', 'abierta', 'abogada', 'aborto', 'abran', 'abrazo', 'abre', 'abrir', 'abuela', 'abuelo', 'abuso', 'abuso violación', 'aca', 'acaba', 'acaban', 'acabar', 'acabará', 'acabo', 'acaso', 'aceptar']


###  RANDOM FOREST

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight='balanced',  
    n_jobs=-1
)

rf_model.fit(X_train_tfidf, y_train)

y_pred_rf = rf_model.predict(X_test_tfidf)
y_pred_proba_rf = rf_model.predict_proba(X_test_tfidf)[:, 1]

print(f" Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f" F1-Score: {f1_score(y_test, y_pred_rf):.4f}")

print("\nREPORTE DE CLASIFICACIÓN:")
print(classification_report(y_test, y_pred_rf, target_names=['No Agresivo', 'Agresivo']))

 Accuracy: 0.7758
 F1-Score: 0.6616

REPORTE DE CLASIFICACIÓN:
              precision    recall  f1-score   support

 No Agresivo       0.83      0.84      0.83       659
    Agresivo       0.67      0.66      0.66       331

    accuracy                           0.78       990
   macro avg       0.75      0.75      0.75       990
weighted avg       0.77      0.78      0.78       990



### LOGISTIC REGRESSION

In [69]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    random_state=42,
    class_weight='balanced'
)

lr_model.fit(X_train_tfidf, y_train)

y_pred_lr = lr_model.predict(X_test_tfidf)

print(f"  Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"  F1-Score: {f1_score(y_test, y_pred_lr):.4f}")

print("REPORTE DE CLASIFICACIÓN:")
print(classification_report(y_test, y_pred_lr, target_names=['No Agresivo', 'Agresivo']))

  Accuracy: 0.7838
  F1-Score: 0.6825
REPORTE DE CLASIFICACIÓN:
              precision    recall  f1-score   support

 No Agresivo       0.84      0.83      0.84       659
    Agresivo       0.67      0.69      0.68       331

    accuracy                           0.78       990
   macro avg       0.76      0.76      0.76       990
weighted avg       0.79      0.78      0.78       990



In [70]:
import joblib
import json
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

if not os.path.exists('../models'):
    os.makedirs('../models')
    print("✓ Carpeta 'models' creada")

modelo_path = f'../models/modelo_agresividad_v1_{timestamp}.pkl'
joblib.dump(lr_model, modelo_path)

vectorizer_path = f'../models/vectorizer_agresividad_v1_{timestamp}.pkl'
joblib.dump(vectorizer, vectorizer_path)

stopwords_path = f'../models/stopwords_agresividad_v1_{timestamp}.json'
with open(stopwords_path, 'w', encoding='utf-8') as f:
    json.dump(stopwords, f, indent=2, ensure_ascii=False)

metadata = {
    'nombre_modelo': 'Logistic Regression',
    'accuracy': 0.7838,
    'f1_score': 0.6825,
    'precision_agresivo': 0.67,
    'recall_agresivo': 0.69,
    'dataset_size': 4950,
    'train_size': 3960,
    'test_size': 990,
    'distribucion': {'No agresivo': 3294, 'Agresivo': 1656},
    'vocabulario_size': 2754,
    'stopwords_size': 647,
    'fecha_entrenamiento': timestamp,
    'hiperparametros': lr_model.get_params(),
    'limitaciones_conocidas': [
        'Falsos negativos en lenguaje sutilmente agresivo',
        'Precisión clase agresivo: 67%',
        'Recall clase agresivo: 69%'
    ],
    'recomendaciones_mejora': [
        'Incorporar análisis léxico específico de odio',
        'Añadir características estilísticas (longitud, puntuación)',
        'Usar dataset_sentimiento.txt para análisis de negatividad'
    ]
}

metadata_path = f'../models/metadata_agresividad_v1_{timestamp}.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

joblib.dump(lr_model, '../models/modelo_agresividad.pkl')
joblib.dump(vectorizer, '../models/vectorizer_agresividad.pkl')

print(f"Modelo guardado: {modelo_path}")
print(f"Vectorizador guardado: {vectorizer_path}")
print(f"Stopwords guardadas: {stopwords_path}")
print(f"Metadata guardada: {metadata_path}")


Modelo guardado: ../models/modelo_agresividad_v1_20251210_0123.pkl
Vectorizador guardado: ../models/vectorizer_agresividad_v1_20251210_0123.pkl
Stopwords guardadas: ../models/stopwords_agresividad_v1_20251210_0123.json
Metadata guardada: ../models/metadata_agresividad_v1_20251210_0123.json


In [71]:
def predecir_agresividad(texto, modelo=lr_model, vectorizador=vectorizer, umbral_confianza=0.65):
    try:
        texto_limpio = preprocesar_texto(texto)
        texto_vectorizado = vectorizador.transform([texto_limpio])
        
        prediccion = modelo.predict(texto_vectorizado)[0]
        probabilidad = modelo.predict_proba(texto_vectorizado)[0]
        confianza = probabilidad[1] if prediccion == 1 else probabilidad[0]
        
        resultado = {
            'texto_original': texto,
            'texto_preprocesado': texto_limpio,
            'prediccion': 'Agresivo' if prediccion == 1 else 'No agresivo',
            'prediccion_num': int(prediccion),
            'confianza': float(confianza),
            'probabilidades': {
                'No agresivo': float(probabilidad[0]),
                'Agresivo': float(probabilidad[1])
            },
            'decisivo': confianza >= umbral_confianza,
            'explicacion': []
        }
        
        if prediccion == 1 and probabilidad[1] > 0.6:
            resultado['explicacion'].append(f"Alta probabilidad de agresividad ({probabilidad[1]:.1%})")
        elif prediccion == 0 and probabilidad[0] > 0.6:
            resultado['explicacion'].append(f"Alta probabilidad de no ser agresivo ({probabilidad[0]:.1%})")
        
        if not resultado['decisivo']:
            resultado['mensaje'] = f'Confianza insuficiente ({confianza:.1%} < {umbral_confianza:.0%}) - Revisión manual recomendada'
        
        return resultado
        
    except Exception as e:
        return {
            'error': str(e),
            'prediccion': 'Error en procesamiento',
            'confianza': 0.0,
            'decisivo': False
        }

In [72]:

ejemplos_prueba = [
    "Odio a todos los inmigrantes, deberían regresar a sus países de mierda",
    "Hoy hace buen clima, me gusta pasear por el parque con mis amigos",
]

for i, ejemplo in enumerate(ejemplos_prueba, 1):
    resultado = predecir_agresividad(ejemplo)
    
    print(f"Ejemplo {i}:")
    print(f"'{ejemplo}'")
    print(f"Predicción: {resultado['prediccion']} ({resultado['confianza']:.1%} confianza)")
    print(f"Probabilidades: No agresivo {resultado['probabilidades']['No agresivo']:.1%} | Agresivo {resultado['probabilidades']['Agresivo']:.1%}")
    print(f"Decisivo: {resultado['decisivo']}")
    

Ejemplo 1:
'Odio a todos los inmigrantes, deberían regresar a sus países de mierda'
Predicción: Agresivo (59.8% confianza)
Probabilidades: No agresivo 40.2% | Agresivo 59.8%
Decisivo: False
Ejemplo 2:
'Hoy hace buen clima, me gusta pasear por el parque con mis amigos'
Predicción: No agresivo (80.8% confianza)
Probabilidades: No agresivo 80.8% | Agresivo 19.2%
Decisivo: True
